# Inicialização

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score
from torchinfo import summary

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from natsort import natsorted

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
# Vamos definir a pasta que está o dataset
workspace = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\'
root = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\RealWorld\\'

# Lista de usuários e atividades
users = natsorted(os.listdir(root))
tarefas = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']
SAC = ['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running']
posicao = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
posicaopt = ['peito', 'antebraço', 'cabeça', 'canela', 'coxa', 'braço', 'cintura']

In [5]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\realworld views\\'
D = [None] * 7
D[0] = pd.read_csv(pasta+posicao[0]+'_nova50hz.csv')
D[1] = pd.read_csv(pasta+posicao[1]+'_nova50hz.csv')
D[2] = pd.read_csv(pasta+posicao[2]+'_nova50hz.csv')
D[3] = pd.read_csv(pasta+posicao[3]+'_nova50hz.csv')
D[4] = pd.read_csv(pasta+posicao[4]+'_nova50hz.csv')
D[5] = pd.read_csv(pasta+posicao[5]+'_nova50hz.csv')
D[6] = pd.read_csv(pasta+posicao[6]+'_nova50hz.csv')

# Definição dos dados e modelos

In [103]:
X = D[0].values[:,:450] / 30
y = D[0].values[:,-3]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()
X_val = torch.tensor(X_val, dtype=torch.float32, device=device).detach()

In [62]:
class automlpacc(nn.Module):
    def __init__(self, dim):
        super().__init__()
        raw_dim = 450
        self.encoder = torch.nn.Sequential(

            nn.Flatten(start_dim=1),
            nn.Linear(raw_dim, dim),
            # nn.ReLU(),
            # nn.LeakyReLU(),
            nn.Tanh()
        )

        self.decoder = nn.Linear(dim, raw_dim)

    def forward(self, x):
        x = self.encoder(x)
        logits = self.decoder(x)
        return logits

In [161]:
class automlp(nn.Module):
    def __init__(self):
        super().__init__()
        dim = 600
        self.encoder = torch.nn.Sequential(

            nn.Flatten(start_dim=1),
            nn.Linear(900, dim),
            # nn.ReLU(),
            # nn.LeakyReLU(),
            nn.Tanh()
        )

        self.decoder = nn.Linear(dim, 900)

    def forward(self, x):
        x = self.encoder(x)
        logits = self.decoder(x)
        return logits

In [137]:
class automlp2(nn.Module):
    def __init__(self):
        super().__init__()
        dim1 = 600
        dim2 = 600
        self.all_layers = torch.nn.Sequential(

            # Encoder
            nn.Flatten(start_dim=1),
            nn.Linear(900, dim1),
            # nn.ReLU(),
            # nn.LeakyReLU(),
            nn.Tanh(),
            nn.Linear(dim1, dim2),
            nn.Tanh(),

            # Decoder
            nn.Linear(dim2, dim1),
            nn.Tanh(),
            nn.Linear(dim1, 900),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [52]:
model = automlpacc().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 20   # number of epochs to run
batch_size = 150  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
trainLoss = []
valLoss = []

In [64]:
summary(model, input_size=(batch_size, 450))

Layer (type:depth-idx)                   Output Shape              Param #
automlpacc                               [150, 450]                --
├─Sequential: 1-1                        [150, 100]                --
│    └─Flatten: 2-1                      [150, 450]                --
│    └─Linear: 2-2                       [150, 100]                45,100
│    └─Tanh: 2-3                         [150, 100]                --
├─Linear: 1-2                            [150, 450]                45,450
Total params: 90,550
Trainable params: 90,550
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 13.58
Input size (MB): 0.27
Forward/backward pass size (MB): 0.66
Params size (MB): 0.36
Estimated Total Size (MB): 1.29

# Treinamento de modelos automlp de uma camada

In [80]:
lat_dim = [10, 20, 30, 50, 70, 100, 150, 200, 250, 300, 350, 400]
models1 = [None] * len(lat_dim)

In [81]:
for i, dim in enumerate(lat_dim):
    models1[i] = automlpacc(dim).to(device)
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(models1[i].parameters(), lr=0.001)
    n_epochs = 20   # number of epochs to run
    batch_size = 150  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
    trainLoss = []
    valLoss = []
    for epoch in range(n_epochs):
        models1[i].train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size,:]
                # forward pass
                Xh = models1[i](X_batch)
                loss = loss_fn(Xh, X_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                bar.set_postfix(loss=float(loss))
        # evaluate accuracy at end of each epoch
        models1[i].eval()
        Xh = models1[i](X_val)
        valLoss.append(float(loss_fn(Xh, X_val)))
        Xh = models1[i](X_train)
        trainLoss.append(float(loss_fn(Xh, X_train)))
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
    fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
    fig.show()

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 215.29batch/s, loss=0.0032] 


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 218.03batch/s, loss=0.00247]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 233.26batch/s, loss=0.00206]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 169.38batch/s, loss=0.00143]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 138.92batch/s, loss=0.000982]


...

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 111.19batch/s, loss=0.000159]


Epoch 19: 100%|██████████| 109/109 [00:01<00:00, 100.00batch/s, loss=7.55e-5] 


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 187.55batch/s, loss=4.34e-5] 


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 186.62batch/s, loss=2.94e-5] 


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 187.89batch/s, loss=1.74e-5]


In [87]:
valLoss = []
for model in models1:
    model.eval()
    Xh = model(X_val)
    valLoss.append(float(loss_fn(Xh, X_val)))
px.bar(x=lat_dim, y=valLoss, title='MSE de reconstrução por dimensão', labels={'y':'MSE', 'x':'Dimensão'}, width=1000)

In [89]:
aux = X_val.cpu().detach().numpy()
i = 12
print(tarefas[int(y_val[i])])
fig = go.Figure()
fig.add_trace(go.Scatter(y=aux[i,:], mode="lines", showlegend=True, name='original'))
for model in models1:
    Xh = model(X_val).cpu().detach().numpy()
    fig.add_trace(go.Scatter(y=Xh[i,:], mode="lines", showlegend=True, name='reconstrução '+str(lat_dim[models1.index(model)])))
fig.show()

running


# Treinamento de modelos a partir do espaço latente dos automlp

In [90]:
emb_train = [None] * len(lat_dim)
emb_val = [None] * len(lat_dim)
for i, model in enumerate(models1):
    emb_train[i] = model.encoder(X_train).detach()
    emb_val[i] = model.encoder(X_val).detach()

In [91]:
lat_dim = [10, 20, 30, 50, 70, 100, 150, 200, 250, 300, 350, 400]
models2 = [None] * len(lat_dim)

In [98]:
class automlp2(nn.Module):
    def __init__(self, dim1, dim2):
        super().__init__()
        self.encoder = torch.nn.Sequential(

            nn.Flatten(start_dim=1),
            nn.Linear(dim1, dim2),
            # nn.ReLU(),
            # nn.LeakyReLU(),
            nn.Tanh()
        )

        self.decoder = nn.Linear(dim2, dim1)

    def forward(self, x):
        x = self.encoder(x)
        logits = self.decoder(x)
        return logits

In [107]:
lossMat = np.zeros((len(lat_dim), len(lat_dim)))
for i, dim1 in enumerate(lat_dim):
    X_train = emb_train[i]
    X_val = emb_val[i]
    for j, dim2 in enumerate(lat_dim):
        models2[j] = automlp2(dim1, dim2).to(device)
        loss_fn = nn.MSELoss()
        optimizer = optim.Adam(models2[j].parameters(), lr=0.001)
        n_epochs = 20   # number of epochs to run
        batch_size = 150  # size of each batch
        batch_start = torch.arange(0, len(X_train), batch_size)
        trainLoss = []
        valLoss = []
        for epoch in range(n_epochs):
            models1[i].train()
            with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
                bar.set_description(f"Epoch {epoch}")
                for start in bar:
                    # take a batch
                    X_batch = X_train[start:start+batch_size,:]
                    # forward pass
                    Xh = models2[j](X_batch)
                    loss = loss_fn(Xh, X_batch)
                    # backward pass
                    optimizer.zero_grad()
                    loss.backward()
                    # update weights
                    optimizer.step()
                    # print progress
                    bar.set_postfix(loss=float(loss))
            # evaluate accuracy at end of each epoch
            models2[j].eval()
            Xh = models2[j](X_val)
            valLoss.append(float(loss_fn(Xh, X_val)))
            Xh = models2[j](X_train)
            trainLoss.append(float(loss_fn(Xh, X_train)))
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
        fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
        fig.show()
    valLoss = []
    for model in models2:
        model.eval()
        Xh = model(X_val)
        valLoss.append(float(loss_fn(Xh, X_val)))
    lossMat[:,i] = np.array(valLoss)
    # px.bar(x=lat_dim, y=valLoss, title='MSE de reconstrução por dimensão', labels={'y':'MSE', 'x':'Dimensão'}, width=1000)

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 180.87batch/s, loss=3.32e-5]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 201.03batch/s, loss=1.56e-5]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 172.76batch/s, loss=1.08e-5]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 156.60batch/s, loss=2.48e-6]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 154.86batch/s, loss=3.52e-6]


...

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 171.19batch/s, loss=0.000821]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 135.01batch/s, loss=0.000403]


Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 122.53batch/s, loss=0.000199]


Epoch 8:  72%|███████▏  | 78/109 [00:00<00:00, 135.82batch/s, loss=0.000239]


KeyboardInterrupt: 

In [109]:
lossMat.round(6)

array([[5.400e-05, 2.443e-03, 3.703e-03, 5.115e-03, 5.555e-03, 5.761e-03,
        5.561e-03, 5.330e-03, 5.024e-03, 4.752e-03, 0.000e+00, 0.000e+00],
       [2.600e-05, 4.800e-05, 1.159e-03, 2.799e-03, 3.653e-03, 4.210e-03,
        4.434e-03, 4.394e-03, 4.167e-03, 3.923e-03, 0.000e+00, 0.000e+00],
       [1.500e-05, 1.800e-05, 3.400e-05, 1.560e-03, 2.540e-03, 3.266e-03,
        3.735e-03, 3.806e-03, 3.666e-03, 3.485e-03, 0.000e+00, 0.000e+00],
       [4.000e-06, 8.000e-06, 9.000e-06, 1.300e-05, 9.440e-04, 1.882e-03,
        2.647e-03, 2.908e-03, 2.911e-03, 2.781e-03, 0.000e+00, 0.000e+00],
       [5.000e-06, 5.000e-06, 5.000e-06, 8.000e-06, 1.000e-05, 9.140e-04,
        1.867e-03, 2.264e-03, 2.345e-03, 2.281e-03, 0.000e+00, 0.000e+00],
       [4.000e-06, 3.000e-06, 3.000e-06, 4.000e-06, 5.000e-06, 8.000e-06,
        9.750e-04, 1.509e-03, 1.697e-03, 1.718e-03, 0.000e+00, 0.000e+00],
       [2.000e-06, 2.000e-06, 2.000e-06, 3.000e-06, 4.000e-06, 4.000e-06,
        6.000e-06, 5.930e-04, 9.

In [110]:
px.imshow(lossMat)

In [ ]:
fig = px.imshow(lossMat.transpose(), width=750, zmax = 1, zmin=0,
          title='MSEs',
          labels=dict(x="Dimensão 2", y="Dimensão 1", color="MSE"),
                y=posicaopt,
                x=np.arange(15).astype(str))

for i,r in enumerate(lossMat):
    for k,c in enumerate(r):
        fig.add_annotation(x=i,y=k,
                           text=str(int((c*100).round(0)))+'%',
                           showarrow=False,
                          )
fig.show()

In [171]:
deeper = automlp600().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(deeper.parameters(), lr=0.001)
n_epochs = 20   # number of epochs to run
batch_size = 150  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
trainLoss = []
valLoss = []

In [169]:
# AJUSTAR ESSE TREINAMENTO!!
for epoch in range(n_epochs):
    deeper.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = emb_train[start:start+batch_size,:]
            # forward pass
            Xh = deeper(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    deeper.eval()
    Xh = deeper(emb_val)
    valLoss.append(float(loss_fn(Xh, emb_val)))
    Xh = deeper(emb_train)
    trainLoss.append(float(loss_fn(Xh, emb_train)))

fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 201.96batch/s, loss=2.21e-5]


In [172]:
# AJUSTAR ESSE TREINAMENTO!!
for epoch in range(n_epochs):
    deeper.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = emb_train[start:start+batch_size,:]
            # forward pass
            Xh = deeper(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    deeper.eval()
    Xh = deeper(emb_val)
    valLoss.append(float(loss_fn(Xh, emb_val)))
    Xh = deeper(emb_train)
    trainLoss.append(float(loss_fn(Xh, emb_train)))

fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 157.77batch/s, loss=6.24e-5] 


# Que mais?